## Pinecone

In [45]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [46]:
pip install -q pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [47]:
pip install --upgrade -q pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [48]:
pip show pinecone-client

Name: pinecone-client
Version: 5.0.1
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /Users/adi2/Library/Python/3.12/lib/python/site-packages
Requires: certifi, pinecone-plugin-inference, pinecone-plugin-interface, tqdm, typing-extensions, urllib3
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [64]:
pip install "pinecone-client[grpc]"


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 6.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [69]:
import os

pinecone_api_key = os.getenv("PINECONE_API_KEY")
print(f"Pinecone API Key: {pinecone_api_key}")  # Debug line


Pinecone API Key: pcsk_6ZB9HU_2Jxhn6bRfk7EfEc9b1dkysGKGNLGmKWzFinUcAvt2Bp1SPeTPiWKJHtspfH6y7f


In [70]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api_key)
print(pc.list_indexes())


{'indexes': []}


In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=pinecone_api_key)

pc.create_index(
  name="example-index1",
  dimension=1536,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1",
  ),
  deletion_protection="disabled"
)

In [90]:
pc.list_indexes()[0]['name']

'example-index1'

In [92]:
pc.describe_index('example-index1')

{'deletion_protection': 'disabled',
 'dimension': 1536,
 'host': 'example-index1-908qlj6.svc.aped-4627-b74a.pinecone.io',
 'metric': 'cosine',
 'name': 'example-index1',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

In [93]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

## Working with Vectors

In [95]:
#inserting vectors
import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
#print(vectors)
ids=list('abcde')
index_name = 'example-index1'
index= pc.Index(index_name)

index.upsert(vectors=zip(ids, vectors))

upserted_count: 5

In [97]:
#updating vectors
index.upsert(vectors=[('c', [0.5] * 1536)])

upserted_count: 1

In [99]:
#fetching vectors
#index = pc.Index(index_name)
#index.fetch(ids=['c', 'd'])

In [100]:
#deleting vectors
index.delete(ids=['b', 'c'])

In [101]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 3}},
 'total_vector_count': 3}

In [102]:
index.fetch(ids=['x'])

{'namespace': '', 'usage': {'read_units': 1}, 'vectors': {}}

In [103]:
#query
query_vector = [random.random() for _ in range(1536)]

In [105]:
index.query(
    vector=query_vector,
    top_k=3,
    include_values=False
)

{'matches': [{'id': 'd',
              'metadata': None,
              'score': 0.7483395,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': 'a',
              'metadata': None,
              'score': 0.7481721,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': 'e',
              'metadata': None,
              'score': 0.7358668,
              'sparse_values': {'indices': [], 'values': []},
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

## Namespaces

In [106]:
index = pc.Index('langchain')
import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
ids = list('abcde')
index.upsert(vectors=zip(ids, vectors))

upserted_count: 5

In [109]:
vectors = [[random.random() for _ in range(1536)] for v in range(3)]
ids = list('xyz')
index.upsert(vectors=zip(ids, vectors), namespace='first-namespace')

upserted_count: 3

In [110]:
vectors = [[random.random() for _ in range(1536)] for v in range(2)]
ids = list('qp')
index.upsert(vectors=zip(ids, vectors), namespace='second-namespace')

upserted_count: 2

In [112]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 8},
                'first-namespace': {'vector_count': 3},
                'second-namespace': {'vector_count': 2}},
 'total_vector_count': 13}

In [117]:
index.fetch(ids=['x'], namespace='first-namespace')

{'namespace': 'first-namespace', 'usage': {'read_units': 1}, 'vectors': {}}

In [116]:
index.delete(ids=['x'], namespace='first-namespace')

In [118]:
index.delete(delete_all=True, namespace='first-namespace')

In [119]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 8},
                'second-namespace': {'vector_count': 2}},
 'total_vector_count': 10}